<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello!

If you're seeing this on GitHub, you can open the document on Google by clicking the "Open in Collab" button above.  Once this document is open in Google, you can see the interactive content by selecting `Runtime` on the menubar and then `Run All.`


# Set up environment


In [5]:
import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap

#Values.GOLD

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-1hc_9y17/mightylogic_1ae62159c0b447a096b4b40a02a0dc96
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-1hc_9y17/mightylogic_1ae62159c0b447a096b4b40a02a0dc96
  Created wheel for MightyLogic: filename=MightyLogic-0.1-py3-none-any.whl size=7772 sha256=e944374f6ef0aba5f6da92eb2615ddf1037078134758d9e98af4cac7ecaedffd
  Stored in directory: /tmp/pip-ephem-wheel-cache-fqlkzbby/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic
  Attempting uninstall: MightyLogic
    Found existing installation: MightyLogic 0.1
    Uninstalling MightyLogic-0.1:
      Successfully uninstalled MightyLogic-0.1


# Turf War Demo

In [7]:

#
# A Turf War Map
#

map2 =TurfWarMap()

# Use convenience functions to build value array
map2.addTile('A',1, mountains(3000,2000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000, 2000))

map2.addTile('B',1, factory(1650,500,5000))
map2.addTile('B',2, mountains(5000,5000))
map2.addTile('B',3, lighthouse(3500,300,7000))
map2.addTile('B',4, plains(5000,5000))
map2.addTile('B',5, mountains(5000,5000))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(5000,5000))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, hells_gate(600,800,8500))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, witch_hut(3500,600, 7000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, altar(600,800,8500))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, lighthouse(2000, 400, 5000))

map2.addTile('E',1, plains(3000, 2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, forest(3000,0,2000))
map2.addTile('E',4, plains(3000, 2000))
map2.addTile('E',5, mountains(3000,2000))
map2.addTile('E',6, forest(3000,0,2000))


# Payout array
image = map2.getValues()
res = map2.stagingScores()
#pprint(image)

In [8]:
#
# An approach that shows well-labeled heatmaps.
#

def show_heatmap(arr, title):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  z_text = flip.round( decimals=-1)

  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  fig.show()



In [9]:
#
# The payout array can be accessed as a Pandas dataframe.
#
payout = pd.DataFrame(image, index=["A","B","C","D","E"], columns=[1,2,3,4,5,6])
payout

,1,2,3,4,5,6
A,30003.0,30003.0,30003.0,15001.6,30003.0,30003.0
B,82582.5,75005.0,109535.0,75005.0,75005.0,136500.0
C,30003.0,75005.0,75005.0,75005.0,139560.0,30003.0
D,30003.0,114035.0,75005.0,139560.0,75005.0,81020.0
E,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0


In [10]:
#
# Heatmaps show the best tiles for payout round and non-payout rounds.
#
show_heatmap(image,"Payout Values for 3rd Rounds")
show_heatmap(res,"Strategic Values for Non-Payout Rounds")

In [11]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('B',6) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                 4,500 gold, 250 contribution
4-6                 3,600 gold, 200 contribution
7-11                2,700 gold, 150 contribution
12-25               1,800 gold, 100 contribution
26-44               900 gold, 50 contribution



# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb

